In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import ast
from scipy.stats import ttest_ind
import seaborn as sns

In [2]:
parquet_file = 'C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/LG/데이터/postech_company_project.parquet'
# Parquet 파일 열기
table = pq.read_table(parquet_file)

# Table에서 데이터를 DataFrame으로 변환
df = table.to_pandas()
print(df.shape)
df.head

,YEAR_YYYY,MONTH_YYYYMM,WEEK_YYYYWWW,RECLAIM_PERIOD,SUBSIDIARY_NAME,CAPTIVE_CHANNEL_FLAG,COMPANY_CODE,DIVISION_NAME,MODEL_CODE,PARTS_NO1,...,KEY_PARTS_REMARK3,KEY_PARTS_REMARK4,KEY_PARTS_REMARK5,PREV_KEY_PARTS_REMARK1,PREV_KEY_PARTS_REMARK2,PREV_KEY_PARTS_REMARK3,PREV_KEY_PARTS_REMARK4,PREV_KEY_PARTS_REMARK5,MULTICLAIM,PREV_RECEIPT_SYMPTOM
0,2022,202201,2022W05,47.0,LGEAI,Non Captive,HE,LTV,OLED65C7P,LGE_PARTS_993,...,Power,None,None,None,None,None,None,None,2,Power
1,2022,202201,2022W03,10.0,LGEAI,Non Captive,HE,LTV,65UN6955ZUF,LGE_PARTS_4492,...,None,None,None,Main,None,None,None,None,2,Audio
2,2022,202201,2022W05,30.0,LGEAI,Non Captive,HE,LTV,50UN7300AUD,LGE_PARTS_15608,...,None,None,None,Main,None,None,None,None,2,Power
3,2022,202201,2022W02,39.0,LGEAI,Non Captive,HE,LTV,70UP7070PUE,LGE_PARTS_9199,...,None,None,None,Power,Main,None,None,None,2,Power
4,2022,202201,2022W03,5.0,LGEAI,Non Captive,HE,LTV,OLED77CXAUA,LGE_PARTS_15413,...,None,None,None,None,None,None,None,None,2,Power


In [8]:
df1 = df[['DIVISION_NAME','PRODUCT_NAME','PRODUCT_NAME_DETAIL']]
df1

,DIVISION_NAME,PRODUCT_NAME,PRODUCT_NAME_DETAIL
0,LTV,OLED TV,OLED TV 65 (UD)
1,LTV,LED LCD TV,LED LCD TV 65 (UD)
2,LTV,LED LCD TV,LED LCD TV 50 (UD)
3,LTV,LED LCD TV,LED LCD TV 70 (UD)
4,LTV,OLED TV,OLED TV 77 (UD)
...,...,...,...
14196328,RAC DIVISION,SRAC,Inverter Single Split Wall_Set
14196329,RAC DIVISION,SRAC,Inverter Single Split Wall_Set
14196330,RAC DIVISION,SRAC,Inverter Single Split Wall_Set
14196331,RAC DIVISION,SRAC,Inverter Single Split Wall_Set


In [9]:
df1 = df1.groupby(['DIVISION_NAME', 'PRODUCT_NAME', 'PRODUCT_NAME_DETAIL']).size().reset_index(name='count')
df1 = df1.iloc[::-1].reset_index(drop=True)
df1

,DIVISION_NAME,PRODUCT_NAME,PRODUCT_NAME_DETAIL,count
0,W/M,Clothes Washer,Clothes Washer_Twin Tub_Pulsator,480710
1,W/M,Clothes Washer,Clothes Washer_Twin Tub_Manual,3985
2,W/M,Clothes Washer,Clothes Washer_Turbo Drum,1932741
3,W/M,Clothes Washer,Clothes Washer_Pulsator(Auto),73075
4,W/M,Clothes Washer,Clothes Washer_Mini Wash,1821
...,...,...,...,...
624,AV,DVD-P,DVD-P+2.1CH SPK,1
625,AV,DVD-P,DVD-P 5Disc Step-Up,1
626,AV,DVD-P,DVD-P 1Disc Step-Up,1
627,AV,DVD-P,DVD-P 1Disc,129


## df2

In [33]:
df2 = df[['DIVISION_NAME','PRODUCT_NAME','PRODUCT_NAME_DETAIL','PARTS_NO1','PARTS_NO2','PARTS_NO3','PARTS_NO4','PARTS_NO5']]
df2

,DIVISION_NAME,PRODUCT_NAME,PRODUCT_NAME_DETAIL,PARTS_NO1,PARTS_NO2,PARTS_NO3,PARTS_NO4,PARTS_NO5
0,LTV,OLED TV,OLED TV 65 (UD),LGE_PARTS_993,LGE_PARTS_18111,LGE_PARTS_20442,None,None
1,LTV,LED LCD TV,LED LCD TV 65 (UD),LGE_PARTS_4492,None,None,None,None
2,LTV,LED LCD TV,LED LCD TV 50 (UD),LGE_PARTS_15608,None,None,None,None
3,LTV,LED LCD TV,LED LCD TV 70 (UD),LGE_PARTS_9199,None,None,None,None
4,LTV,OLED TV,OLED TV 77 (UD),LGE_PARTS_15413,None,None,None,None
...,...,...,...,...,...,...,...,...
14196328,RAC DIVISION,SRAC,Inverter Single Split Wall_Set,None,None,None,None,None
14196329,RAC DIVISION,SRAC,Inverter Single Split Wall_Set,None,None,None,None,None
14196330,RAC DIVISION,SRAC,Inverter Single Split Wall_Set,None,None,None,None,None
14196331,RAC DIVISION,SRAC,Inverter Single Split Wall_Set,None,None,None,None,None


In [34]:
print('parts1이 존재하는 확률',len(df2[df2['PARTS_NO1'].notna()])/len(df2)*100)
print('parts2이 존재하는 확률',len(df2[df2['PARTS_NO2'].notna()])/len(df2)*100)
print('parts3이 존재하는 확률',len(df2[df2['PARTS_NO3'].notna()])/len(df2)*100)
print('parts4이 존재하는 확률',len(df2[df2['PARTS_NO4'].notna()])/len(df2)*100)
print('parts5이 존재하는 확률',len(df2[df2['PARTS_NO5'].notna()])/len(df2)*100)

parts1이 존재하는 확률 36.43152895654321
parts2이 존재하는 확률 8.162825571467309
parts3이 존재하는 확률 3.051374786367025
parts4이 존재하는 확률 1.444751076341353
parts5이 존재하는 확률 0.7013400201307026


In [29]:
product_name_list = df1['PRODUCT_NAME'].unique()
product_name_list

array(['Clothes Washer', 'Clothes Styler', 'Clothes Dryer',
       'Accessory & Parts', 'Vacuum Cleaner', 'Commercial_Signage Board',
       'Commercial_OLED Monitor', 'Commercial_Monitor Consumer Panel',
       'Commercial_Monitor Accessory', 'Commercial_LED Signage',
       'Commercial_LED LCD Monitor', 'Commercial_LCD Monitor', 'Security',
       'REF Comp.', 'TMNF', 'SxS', 'Specific REF', 'REF Parts',
       'REF Kits', 'Gimchi REF', 'Built-In REF', 'Bottom Freezer', 'WRAC',
       'SRAC', 'PAC_Slim', 'Airconditioner Acessory & Others',
       'A/C Mock-up', 'Projector ', 'PDP TV', 'Commercial_PDP TV',
       'Notebook PC', 'OLED Monitor', 'Medical Equipment', 'LCD Monitor',
       'FST Monitor', 'FCDT Monitor', 'Cloud Device', 'TV Accessory',
       'Projection TV', 'Premium OLED TV', 'OLED TV', 'LED LCD TV',
       'LCD TV', 'Display Parts(TV)', 'Commercial_LCD TV',
       'Wellness care', 'Dishwasher', 'Range', 'Oven',
       'Other CookingAppliance', 'MWO', 'LWO', 'Hood', 'HOB'

In [35]:
# 새로운 열을 추가할 빈 리스트 생성
new_column_values = []

# df1의 PRODUCT_NAME을 하나씩 확인
for product_name in df1['PRODUCT_NAME'].unique():
    # df2에서 해당 PRODUCT_NAME에 대한 행 선택
    subset_df2 = df2[df2['PRODUCT_NAME'] == product_name]

    # PARTS_NO1부터 PARTS_NO5까지의 값을 가져와서 중복을 제외하고 결합
    parts_values = subset_df2[['PARTS_NO1', 'PARTS_NO2', 'PARTS_NO3', 'PARTS_NO4', 'PARTS_NO5']].stack().unique()
    parts_values_str = ','.join(map(str, parts_values))

    # 새로운 열에 기록
    new_column_values.append(parts_values_str)

# df1에 새로운 열 추가
df1['PARTS_ALL'] = np.nan

# df1과 product_name_list를 활용하여 새로운 열에 값 기록
for idx, product_name in enumerate(df1['PRODUCT_NAME'].unique()):
    mask = df1['PRODUCT_NAME'] == product_name
    df1.loc[mask, 'PARTS_ALL'] = new_column_values[idx]

C:\Users\jjccyy9741\AppData\Local\Temp\ipykernel_17416\2808746416.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_14075,LGE_PARTS_14058,LGE_PARTS_5605,LGE_PARTS_19527,LGE_PARTS_2288,LGE_PARTS_19632,LGE_PARTS_31549,LGE_PARTS_9304,LGE_PARTS_6937,LGE_PARTS_17680,LGE_PARTS_33883,LGE_PARTS_13625,LGE_PARTS_20605,LGE_PARTS_21099,LGE_PARTS_28301,LGE_PARTS_2787,LGE_PARTS_4616,LGE_PARTS_26749,LGE_PARTS_12100,LGE_PARTS_10907,LGE_PARTS_27837,LGE_PARTS_30364,LGE_PARTS_4952,LGE_PARTS_17228,LGE_PARTS_18646,LGE_PARTS_33902,LGE_PARTS_18786,LGE_PARTS_14776,LGE_PARTS_28417,LGE_PARTS_24719,LGE_PARTS_14120,LGE_PARTS_33744,LGE_PARTS_32176,LGE_PARTS_30260,LGE_PARTS_15328,LGE_PARTS_895,LGE_PARTS_21659,LGE_PARTS_29181,LGE_PARTS_20697,LGE_PARTS_6552,LGE_PARTS_34990,LGE_PARTS_29466,LGE_PARTS_8712,LGE_PARTS_13694,LGE_PARTS_24310,LGE_PARTS_21305,LGE_PARTS_13029,LGE_PARTS_8257,LGE_PARTS_34931,LGE_P

In [36]:
df1

,DIVISION_NAME,PRODUCT_NAME,PRODUCT_NAME_DETAIL,count,PARTS_ALL
0,W/M,Clothes Washer,Clothes Washer_Twin Tub_Pulsator,480710,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407..."
1,W/M,Clothes Washer,Clothes Washer_Twin Tub_Manual,3985,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407..."
2,W/M,Clothes Washer,Clothes Washer_Turbo Drum,1932741,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407..."
3,W/M,Clothes Washer,Clothes Washer_Pulsator(Auto),73075,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407..."
4,W/M,Clothes Washer,Clothes Washer_Mini Wash,1821,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407..."
...,...,...,...,...,...
624,AV,DVD-P,DVD-P+2.1CH SPK,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058..."
625,AV,DVD-P,DVD-P 5Disc Step-Up,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058..."
626,AV,DVD-P,DVD-P 1Disc Step-Up,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058..."
627,AV,DVD-P,DVD-P 1Disc,129,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058..."


In [37]:
# 새로운 열을 추가할 빈 리스트 생성
new_columns_values = {f'PARTS{i}': [] for i in range(1, 6)}

# df1의 PRODUCT_NAME을 하나씩 확인
for product_name in df1['PRODUCT_NAME'].unique():
    # df2에서 해당 PRODUCT_NAME에 대한 행 선택
    subset_df2 = df2[df2['PRODUCT_NAME'] == product_name]

    # PARTS_NO1부터 PARTS_NO5까지의 유니크한 값을 가져와서 새로운 열에 기록
    for i in range(1, 6):
        parts_column = f'PARTS_NO{i}'
        unique_parts_values = subset_df2[parts_column].dropna().unique()
        new_columns_values[f'PARTS{i}'].append(','.join(map(str, unique_parts_values)))

# df1에 새로운 열 추가
for i in range(1, 6):
    df1[f'PARTS{i}'] = np.nan

# df1과 new_columns_values를 활용하여 새로운 열에 값 기록
for i in range(1, 6):
    for idx, product_name in enumerate(df1['PRODUCT_NAME'].unique()):
        mask = df1['PRODUCT_NAME'] == product_name
        df1.loc[mask, f'PARTS{i}'] = new_columns_values[f'PARTS{i}'][idx]


C:\Users\jjccyy9741\AppData\Local\Temp\ipykernel_17416\1740876024.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_14058,LGE_PARTS_19527,LGE_PARTS_2288,LGE_PARTS_19632,LGE_PARTS_6937,LGE_PARTS_13625,LGE_PARTS_21099,LGE_PARTS_2787,LGE_PARTS_10907,LGE_PARTS_27837,LGE_PARTS_14776,LGE_PARTS_28417,LGE_PARTS_33744,LGE_PARTS_32176,LGE_PARTS_24719,LGE_PARTS_895,LGE_PARTS_21659,LGE_PARTS_29181,LGE_PARTS_34990,LGE_PARTS_14075,LGE_PARTS_13694,LGE_PARTS_24310,LGE_PARTS_21305,LGE_PARTS_8257,LGE_PARTS_34484,LGE_PARTS_32755,LGE_PARTS_3237,LGE_PARTS_10221,LGE_PARTS_3260,LGE_PARTS_825,LGE_PARTS_12248,LGE_PARTS_13629,LGE_PARTS_13723,LGE_PARTS_12360,LGE_PARTS_35143,LGE_PARTS_32100,LGE_PARTS_8654,LGE_PARTS_5632,LGE_PARTS_18228,LGE_PARTS_15395,LGE_PARTS_26749,LGE_PARTS_5605,LGE_PARTS_19623,LGE_PARTS_362,LGE_PARTS_25040,LGE_PARTS_26187,LGE_PARTS_27296,LGE_PARTS_22534,LGE_PARTS_25967,LGE_PART

In [38]:
df1

,DIVISION_NAME,PRODUCT_NAME,PRODUCT_NAME_DETAIL,count,PARTS_ALL,PARTS1,PARTS2,PARTS3,PARTS4,PARTS5
0,W/M,Clothes Washer,Clothes Washer_Twin Tub_Pulsator,480710,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407...","LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1405...","LGE_PARTS_14075,LGE_PARTS_5605,LGE_PARTS_31549...","LGE_PARTS_29914,LGE_PARTS_9304,LGE_PARTS_33883...","LGE_PARTS_12100,LGE_PARTS_17228,LGE_PARTS_1878...","LGE_PARTS_23233,LGE_PARTS_3365,LGE_PARTS_17042..."
1,W/M,Clothes Washer,Clothes Washer_Twin Tub_Manual,3985,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407...","LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1405...","LGE_PARTS_14075,LGE_PARTS_5605,LGE_PARTS_31549...","LGE_PARTS_29914,LGE_PARTS_9304,LGE_PARTS_33883...","LGE_PARTS_12100,LGE_PARTS_17228,LGE_PARTS_1878...","LGE_PARTS_23233,LGE_PARTS_3365,LGE_PARTS_17042..."
2,W/M,Clothes Washer,Clothes Washer_Turbo Drum,1932741,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407...","LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1405...","LGE_PARTS_14075,LGE_PARTS_5605,LGE_PARTS_31549...","LGE_PARTS_29914,LGE_PARTS_9304,LGE_PARTS_33883...","LGE_PARTS_12100,LGE_PARTS_17228,LGE_PARTS_1878...","LGE_PARTS_23233,LGE_PARTS_3365,LGE_PARTS_17042..."
3,W/M,Clothes Washer,Clothes Washer_Pulsator(Auto),73075,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407...","LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1405...","LGE_PARTS_14075,LGE_PARTS_5605,LGE_PARTS_31549...","LGE_PARTS_29914,LGE_PARTS_9304,LGE_PARTS_33883...","LGE_PARTS_12100,LGE_PARTS_17228,LGE_PARTS_1878...","LGE_PARTS_23233,LGE_PARTS_3365,LGE_PARTS_17042..."
4,W/M,Clothes Washer,Clothes Washer_Mini Wash,1821,"LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1407...","LGE_PARTS_28040,LGE_PARTS_29914,LGE_PARTS_1405...","LGE_PARTS_14075,LGE_PARTS_5605,LGE_PARTS_31549...","LGE_PARTS_29914,LGE_PARTS_9304,LGE_PARTS_33883...","LGE_PARTS_12100,LGE_PARTS_17228,LGE_PARTS_1878...","LGE_PARTS_23233,LGE_PARTS_3365,LGE_PARTS_17042..."
...,...,...,...,...,...,...,...,...,...,...
624,AV,DVD-P,DVD-P+2.1CH SPK,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_8121,LGE_PARTS_20301",,,
625,AV,DVD-P,DVD-P 5Disc Step-Up,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_8121,LGE_PARTS_20301",,,
626,AV,DVD-P,DVD-P 1Disc Step-Up,1,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_8121,LGE_PARTS_20301",,,
627,AV,DVD-P,DVD-P 1Disc,129,"LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_21824,LGE_PARTS_12852,LGE_PARTS_3058...","LGE_PARTS_8121,LGE_PARTS_20301",,,


In [43]:
df1.to_excel('C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/LG/csv파일/제품세분화2.xlsx')
